In [1]:
import os
import random
import numpy as np
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

# EDA

In [2]:
import json
import pandas as pd

In [3]:
path = r"C:\Users\esual\Downloads\seminar_fma_cl\data\eli5"
file = 'generate_a_question_1.train_asks.json'

# Load data
with open(os.path.join(path, file), 'r') as f:
    data = json.load(f)

In [4]:
# Convert to DataFrame
df = pd.DataFrame(data)
df.head()

,src,tgt,src_info
0,I did some research as a layman. One forum use...,What are the long-term effects of using raid?,{'question': 'What are the long-term effects o...
1,Marine biologists and geologists often have op...,"Hey reddit, what are some jobs in science that...","{'question': 'Hey reddit, what are some jobs i..."
2,"It's related to the ""beat"" period/frequency - ...","What is the proper name for a period T, when t...",{'question': 'What is the proper name for a pe...
3,Write an inquisitive question about the follow...,When two different versions of the same word c...,{'question': 'When two different versions of t...
4,"Given the following text, what could have been...","How big do people from ""pygmy"" groups get when...","{'question': 'How big do people from ""pygmy"" g..."


In [12]:
df.shape

(61710, 3)

In [14]:
df.iloc[0,'src_info']

src         I did some research as a layman. One forum use...
tgt             What are the long-term effects of using raid?
src_info    {'question': 'What are the long-term effects o...
Name: 0, dtype: object

In [15]:
df.query('tgt == "What are the long-term effects of using raid?"')

,src,tgt,src_info
0,I did some research as a layman. One forum use...,What are the long-term effects of using raid?,{'question': 'What are the long-term effects o...


# Create dataset

In [5]:
path_data = r"C:\Users\esual\Downloads\seminar_fma_cl\data"

In [ ]:
import json
import random
random.seed(666)

mapper = lambda x: 'validation' if x in {'quail', 'social_i_qa', 'quoref', 'validation'} else 'test'

#d_t0_train_datasets = {name: {mapper(name): [('t0_template', "__RANDOM__")]} for name in t0_train_datasets if name not in skip}
#process_datasets(d_t0_train_datasets, limit_nb_examples=1000, path_data=path_data)

all_data = {'src':[], 'tgt':[], 'src_info':[]}
for name_dataset in [
        'qasc', 'quarel', 'sciq', 'cosmos_qa', 'wiqa', 'social_i_qa', 'quartz',  'quail', 'adversarial_qa_dbert', 'adversarial_qa_dbidaf',
        'adversarial_qa_droberta', 'duorc_ParaphraseRC', 'duorc_SelfRC', 'quoref', 'wiki_qa', 'common_gen', 'imdb', 'xsum', 'ag_news',
        'rotten_tomatoes', 'glue_qqp', 'trec', 'paws_labeled_final', 'glue_mrpc', 'dream'
  ]:

  path = f"{path_data}/{name_dataset}/__RANDOM__.{mapper(name_dataset)}.json"

  with open(path, 'r') as f:
    data = json.load(f)

    

  for k in ['src', 'tgt', 'src_info']:
    all_data[k] += data[k]

sampled_ids = random.sample(list(range(len(all_data[k]))), k=1000)

sampled_data = {}
for k in ['src', 'tgt', 'src_info']:
  sampled_data[k] = [all_data[k][i] for i in sampled_ids]

path = f"{path_data}/T0_train_eval/__RANDOM__.test.json"
with open(path, 'w') as f:
  json.dump(sampled_data, f)

In [14]:
all_data['src_info'][10000]

{'id': 'bb7faa8846f5bf96f63e956d8b0dccaa36fd2236',
 'title': 'Yuan_dynasty',
 'context': 'Many Han Chinese and Khitan defected to the Mongols to fight against the Jin. Two Han Chinese leaders, Shi Tianze, Liu Heima (劉黑馬, Liu Ni), and the Khitan Xiao Zhala (蕭札剌) defected and commanded the 3 Tumens in the Mongol army. Liu Heima and Shi Tianze served Ogödei Khan. Liu Heima and Shi Tianxiang led armies against Western Xia for the Mongols. There were 4 Han Tumens and 3 Khitan Tumens, with each Tumen consisting of 10,000 troops. The three Khitan Generals Shimobeidier (石抹孛迭兒), Tabuyir (塔不已兒) and Xiaozhacizhizizhongxi (蕭札刺之子重喜) commanded the three Khitan Tumens and the four Han Generals Zhang Rou, Yan Shi, Shi Tianze, and Liu Heima commanded the four Han tumens under Ogödei Khan.',
 'question': 'Who was lower in rank, Ogodei Khan or Liu Heima?',
 'answers': {'text': [], 'answer_start': []},
 'metadata': {'split': 'test', 'model_in_the_loop': 'RoBERTa-Large'},
 'prompt_name': 'generate_question

In [16]:
len(all_data['src'])

23790

In [17]:
mapper = lambda x: 'validation' if x == 'asset' else ('train_asks' if x == 'eli5' else 'train')

all_data = {'src':[], 'tgt':[], 'src_info':[]}
for name_dataset in [
        'wiki_auto', # Simp
        'asset', # Simp
        'eli5', # InqQG
        'eSNLI', # Exp
        'empathetic_dialogues', # Emdg
        'gigaword' # HGen
  ]:

  path = f"{path_data}/{name_dataset}/__RANDOM__.{mapper(name_dataset)}.json"

  with open(path, 'r') as f:
    data = json.load(f)

    if data is None:
      print(f"Skipping {name_dataset}")
      continue

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\esual\\Downloads\\seminar_fma_cl\\data/wiki_auto/__RANDOM__.train.json'

In [18]:
from t0_continual_learning import model_prediction
from t0_continual_learning.config_variables import evaluation_new_tasks, evaluation_T0evalsets

#path_data = f'{GLOBAL_PATH}/data'
#path_pred = f'{GLOBAL_PATH}/predictions'

d_datasets = {
    **evaluation_new_tasks,
    **evaluation_T0evalsets
}

c:\Users\esual\miniconda3\envs\clearning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
d_datasets.keys()

dict_keys(['gigaword', 'wiki_auto', 'asset', 'eli5', 'haiku', 'covid_qa_deepset', 'empathetic_dialogues', 'eSNLI', 'twitter_top20', 'T0_train_eval', 'winogrande', 'rte', 'wic', 'copa', 'hellaswag', 'anli', 'cb', 'wsc', 'story_cloze'])

In [22]:
d_datasets['wiki_auto']

{'test': {'simplification_1': {'type': 'custom',
   'choice': '',
   'metrics': ['bleu', 'sari']}}}

# only wikiauto

In [1]:
import os
import numpy as np
import json
import torch
import pandas as pd

#import evaluate
from datasets import load_dataset
import random

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM
)

In [2]:
torch.cuda.is_available()

True

In [30]:
name_dataset = 'wiki_auto'
path = f"../data/raw/{name_dataset}/simplification_1.test.json"
with open(path, 'r') as f:
  data = json.load(f)

In [31]:
data

{'src': ['Reformulate this text with simpler words: "His father Robert Alda -LRB- born Alphonso Giuseppe Giovanni Roberto D \'Abruzzo -RRB- was an actor and singer , and his mother Joan Browne was a homemaker and former beauty-pageant winner .\n"',
  'Make this text simpler: "The new colonists named their settlement after the Canton of Bern , Switzerland , home state of their patron .\n"',
  'Make this text simpler: "New Bern is the second-oldest European settled colonial town in North Carolina , after Bath .\n"',
  'It served as the capital of the North Carolina colonial government , then briefly as the state capital .\n\n\n===\n\nGiven the above text, reformulate it so it is easier to read:',
  'Reformulate the following text with simpler words: "After the American Revolution , New Bern became wealthy and quickly developed a rich cultural life .\n"',
  'At one time New Bern was called " the Athens of the South , " renowned for its Masonic Temple and Athens Theater .\n\n\n===\n\nGiven

In [32]:
data['src']

['Reformulate this text with simpler words: "His father Robert Alda -LRB- born Alphonso Giuseppe Giovanni Roberto D \'Abruzzo -RRB- was an actor and singer , and his mother Joan Browne was a homemaker and former beauty-pageant winner .\n"',
 'Make this text simpler: "The new colonists named their settlement after the Canton of Bern , Switzerland , home state of their patron .\n"',
 'Make this text simpler: "New Bern is the second-oldest European settled colonial town in North Carolina , after Bath .\n"',
 'It served as the capital of the North Carolina colonial government , then briefly as the state capital .\n\n\n===\n\nGiven the above text, reformulate it so it is easier to read:',
 'Reformulate the following text with simpler words: "After the American Revolution , New Bern became wealthy and quickly developed a rich cultural life .\n"',
 'At one time New Bern was called " the Athens of the South , " renowned for its Masonic Temple and Athens Theater .\n\n\n===\n\nGiven the above te

In [33]:
# Find 'Robert Alda' in the data['src]
for i, src in enumerate(data['src']):
    if 'Alda ' in src:
        print(i, src)
        break
    else:
        print(i, '---')

0 Reformulate this text with simpler words: "His father Robert Alda -LRB- born Alphonso Giuseppe Giovanni Roberto D 'Abruzzo -RRB- was an actor and singer , and his mother Joan Browne was a homemaker and former beauty-pageant winner .
"


In [5]:
data.keys()

dict_keys(['src', 'tgt', 'src_info'])

In [6]:
data_df = pd.DataFrame(data)

In [7]:
rand_ind = random.randint(0, len(data_df))
print(data_df.iloc[rand_ind,2])
print('====================')
print(data_df.iloc[rand_ind,1])
print('====================')
print(data_df.iloc[rand_ind,0])

{'normal_sentence': 'The water depth is 57 m and at 100 % capacity the water level is 344 m AHD .\n', 'simple_sentence': 'Normal water depth is 57 m .\n', 'prompt': '[normal_sentence]\n\n===\n\nGiven this text, write it with, simpler words:', 'prompt_name': 'simplification_1', 'prompt_mode': 'custom'}
Normal water depth is 57 m .

The water depth is 57 m and at 100 % capacity the water level is 344 m AHD .


===

Given this text, write it with, simpler words:


In [8]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
os.environ['TOKENIZERS_PARALLELISM'] = 'false' # there might be interferences with the parallelism of the Hugging Face Trainer

In [9]:
model_name = "llm-stacking/StackLLM_3B_300BToken"  # Replace with the name of your pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)

C:\Users\esual\miniconda3\envs\clearning\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
data_rand = data_df.iloc[rand_ind]

In [11]:
data_rand

src         The water depth is 57 m and at 100 % capacity ...
tgt                            Normal water depth is 57 m .\n
src_info    {'normal_sentence': 'The water depth is 57 m a...
Name: 39520, dtype: object

In [12]:
tokenizer(data_rand["src"], truncation=True, max_length=512)

{'input_ids': [1, 450, 4094, 10809, 338, 29871, 29945, 29955, 286, 322, 472, 29871, 29896, 29900, 29900, 1273, 13284, 278, 4094, 3233, 338, 29871, 29941, 29946, 29946, 286, 319, 26124, 869, 13, 13, 13, 25512, 13, 13, 29954, 5428, 445, 1426, 29892, 2436, 372, 411, 29892, 13682, 3838, 29901], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
tokenizer(data_rand["tgt"], truncation=True, max_length=128)

{'input_ids': [1, 21981, 4094, 10809, 338, 29871, 29945, 29955, 286, 869, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
def preprocess_simpl(data):
    tokenized_data = tokenizer(data['src'], truncation=True, max_length=512)
    tokenized_labels = tokenizer(data['tgt'], truncation=True, max_length=128)
    tokenized_data['labels'] = tokenized_labels['input_ids']
    return tokenized_data

In [15]:
type(data['src'])

list

In [16]:
tokenized_wiki = data_df.apply(preprocess_simpl, axis=1)

In [17]:
tokenized_wiki.iloc[10]

{'input_ids': [1, 23856, 1306, 29871, 30478, 341, 6637, 3219, 493, 1794, 448, 29519, 29933, 29899, 11324, 29871, 30712, 29871, 30511, 29874, 29871, 30712, 29871, 30176, 31513, 30234, 448, 2227, 4162, 29899, 286, 448, 2227, 4162, 29899, 29871, 30426, 30215, 448, 2227, 4162, 29899, 29871, 30176, 448, 2227, 4162, 29899, 260, 448, 2227, 4162, 29899, 263, 30312, 2056, 29871, 29896, 3786, 29871, 29896, 29929, 29946, 29900, 785, 29871, 29906, 29945, 3839, 29871, 29906, 29900, 29896, 29896, 448, 29934, 29934, 29933, 29899, 471, 263, 4325, 26689, 10015, 10094, 5264, 1919, 29380, 322, 8604, 5039, 391, 322, 278, 937, 11715, 6114, 304, 5401, 278, 27813, 20604, 869, 13, 13, 13, 25512, 13, 13, 29954, 5428, 278, 2038, 1426, 29892, 2436, 372, 411, 29892, 13682, 3838, 29901], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [18]:
model = AutoModelForCausalLM.from_pretrained(model_name)

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at llm-stacking/StackLLM_3B_300BToken and are newly initialized: ['model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.9.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.29.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.31.self_attn.rotary_emb.inv_freq', 'model.layers.26.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn

In [19]:
# checking if model exists
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2560, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (k_proj): Linear(in_features=2560, out_features=512, bias=False)
          (v_proj): Linear(in_features=2560, out_features=512, bias=False)
          (o_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2560, out_features=7040, bias=False)
          (up_proj): Linear(in_features=2560, out_features=7040, bias=False)
          (down_proj): Linear(in_features=7040, out_features=2560, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()


In [2]:
test_data = load_dataset('tatsu-lab/alpaca', trust_remote_code=True)

In [3]:
test_data = test_data['train']
test_data

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})

In [12]:
# Choose random sample
sample = random.choice(test_data)

# Print one sample from dataset
print(f'''
Instruction: {sample['instruction']}
Input: {sample['input']}
Output: {sample['output']}
Text: {sample['text']}


''')


Instruction: Summarize the given text in 5-8 sentences.
Input: The German Health Interview and Examination Survey for Children and Adolescents (KiGGS) is one of the largest and most comprehensive surveys ever conducted on the health and lifestyle of young people in Germany. It started in 2003 and looks at the physical and mental health of German children and adolescents between the ages of 0 and 17. In addition to providing insight into the general health of the population, KiGGS also investigates other aspects of health such as physical activity, nutrition, and family environment, as well as socio-economic determinants of health.
Output: The German Health Interview and Examination Survey for Children and Adolescents (KiGGS) is a broad survey that provides insight into the health and lifestyle of young people in Germany. The survey started in 2003 and examines the physical and mental health of children aged 0-17 from all over the country. On top of exploring general health, the survey

In [7]:
sample

{'instruction': 'Find the words in the sentence which suggest a positive attitude',
 'input': 'I think I can do this!',
 'output': '"I think", "can do"',
 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nFind the words in the sentence which suggest a positive attitude\n\n### Input:\nI think I can do this!\n\n### Response:\n"I think", "can do"'}

In [13]:
sampled_data

NameError: name 'sampled_data' is not defined

In [4]:
name_dataset = 'wiki_auto'
path = f"../data/raw/{name_dataset}/simplification_1.test.json"

# read json file with pd.read_json
df = pd.read_json(path)

In [44]:
# read all json files in ../data/raw/wiki_auto and merge them
path = f"../data/raw/{name_dataset}"
all_files = os.listdir(path)
li = []

for filename in all_files:
    if filename.endswith('.json'):
        file_path = os.path.join(path, filename)
        data = pd.read_json(file_path)
        data['source'] = filename
        li.append(data)

df = pd.concat(li, axis=0, ignore_index=True)

In [45]:
# find the row where src contains 'Alda ' 
df[df['src'].str.contains('Alda ')].to_clipboard()

In [46]:
df.head()

,src,tgt,src_info,source
0,"Reformulate this text with simpler words: ""His...","His father , Robert Alda , was an actor , and ...",{'normal_sentence': 'His father Robert Alda -L...,simplification_1.test.json
1,"Make this text simpler: ""The new colonists nam...",It was named after the capital of Switzerland ...,{'normal_sentence': 'The new colonists named t...,simplification_1.test.json
2,"Make this text simpler: ""New Bern is the secon...",New Bern is the second oldest town in North Ca...,{'normal_sentence': 'New Bern is the second-ol...,simplification_1.test.json
3,It served as the capital of the North Carolina...,It was the capital of the North Carolina colon...,{'normal_sentence': 'It served as the capital ...,simplification_1.test.json
4,Reformulate the following text with simpler wo...,"After the American Revolution , New Bern becam...",{'normal_sentence': 'After the American Revolu...,simplification_1.test.json


In [43]:
print('df size:', df.shape)
print('df unique size:', df['tgt'].nunique())

df size: (108002, 3)
df unique size: 102994


In [48]:
df.source.value_counts()

source
simplification_1.train.json    100002
simplification_1.test.json       4000
simplification_2.test.json       4000
Name: count, dtype: int64

In [50]:
df.rename(columns = {'src': 'input', 'tgt': 'output'}, inplace=True)
# save df as json to the same folder
df.to_json(path + '/simplification_man.json')

# checking tokenizers

In [5]:
from transformers import AutoTokenizer

model_path = "llm-stacking/StackLLM_410M_750BToken"
# Load your tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Check if the tokens exist in the vocabulary
special_tokens = ["<<SYS>>", "<</SYS>>", "[INST]", "[/INST]"]
for token in special_tokens:
    if token in tokenizer.get_vocab():
        print(f"'{token}' is in the tokenizer's vocabulary.")
    else:
        print(f"'{token}' is NOT in the tokenizer's vocabulary.")

'<<SYS>>' is NOT in the tokenizer's vocabulary.
'<</SYS>>' is NOT in the tokenizer's vocabulary.
'[INST]' is NOT in the tokenizer's vocabulary.
'[/INST]' is NOT in the tokenizer's vocabulary.


In [8]:
# Example check for -LRB- and -RRB-
tokens_to_check = ["(", ")"]
for token in tokens_to_check:
    token_id = tokenizer.convert_tokens_to_ids(token)
    if token_id == tokenizer.unk_token_id:
        print(f"Token '{token}' is NOT in the tokenizer vocabulary.")
    else:
        print(f"Token '{token}' exists with ID {token_id}.")


Token '(' exists with ID 29898.
Token ')' exists with ID 29897.


In [7]:
# Example check for -LRB- and -RRB-
tokens_to_check = ["LRB", "RRB"]
for token in tokens_to_check:
    token_id = tokenizer.convert_tokens_to_ids(token)
    if token_id == tokenizer.unk_token_id:
        print(f"Token '{token}' is NOT in the tokenizer vocabulary.")
    else:
        print(f"Token '{token}' exists with ID {token_id}.")


Token 'LRB' is NOT in the tokenizer vocabulary.
Token 'RRB' is NOT in the tokenizer vocabulary.


In [4]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
import torch
import transformers
from torch.nn.utils.rnn import pad_sequence
import argparse
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    set_seed,
    Seq2SeqTrainer,
    LlamaTokenizer

)
from typing import Optional, Dict, Sequence
import logging
from tqdm.auto import tqdm
transformers.utils.logging.set_verbosity_info()  # Set logging level to INFO
transformers.utils.logging.enable_progress_bar()  # Enable progress bars

In [4]:
# check gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
if torch.cuda.is_available():   
    torch.backends.cuda.matmul.allow_tf32 = True

logger = logging.getLogger(__name__)

IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"

In [4]:
sample_text = "this is an example."
tokens = tokenizer(sample_text, truncation=True, max_length=512)
tokens

{'input_ids': [1, 445, 338, 385, 1342, 29889], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [5]:
tokenizer.batch_decode(tokens['input_ids'])

['<s>', 'this', 'is', 'an', 'example', '.']

In [7]:
import os
import json
from datasets import Dataset

# Define the file path
dataset_name = "../data/preprocessed/wiki_auto_simp_new.json"
dataset_name = os.path.abspath(dataset_name)

if dataset_name.endswith('.json') or dataset_name.endswith('.jsonl'):
        train_dataset = Dataset.from_json(path_or_paths=dataset_name, field='train')
        eval_dataset = Dataset.from_json(path_or_paths=dataset_name, field='eval')
        test_dataset = Dataset.from_json(path_or_paths=dataset_name, field='test')
        full_dataset = {'train': train_dataset, 'eval': eval_dataset, 'test': test_dataset}

In [8]:
full_dataset

{'train': Dataset({
     features: ['input', 'label', 'src_info', 'index'],
     num_rows: 95001
 }),
 'eval': Dataset({
     features: ['input', 'label', 'src_info', 'index'],
     num_rows: 5001
 }),
 'test': Dataset({
     features: ['input', 'label', 'src_info', 'index'],
     num_rows: 8000
 })}

In [9]:
def smart_tokenizer_and_embedding_resize(
    special_tokens_dict: Dict,
    tokenizer: transformers.PreTrainedTokenizer,
    model: transformers.PreTrainedModel,
    non_special_tokens = None,
):
    """Resize tokenizer and embedding.

    Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
    """
    num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict) + tokenizer.add_tokens(non_special_tokens)
    model.resize_token_embeddings(len(tokenizer))
    
    if num_new_tokens > 0:
        input_embeddings_data = model.get_input_embeddings().weight.data
        output_embeddings_data = model.get_output_embeddings().weight.data

        input_embeddings_avg = input_embeddings_data[:-num_new_tokens].mean(dim=0, keepdim=True)
        output_embeddings_avg = output_embeddings_data[:-num_new_tokens].mean(dim=0, keepdim=True)

        input_embeddings_data[-num_new_tokens:] = input_embeddings_avg
        output_embeddings_data[-num_new_tokens:] = output_embeddings_avg
    print(f"Resized tokenizer and embedding to {len(tokenizer)} tokens.")

In [34]:
class ArgsObject:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

# Example dictionary
args_dict = {
    'special_tokens_dict': {'additional_special_tokens': ["<<SYS>>", "<</SYS>>", "[INST]", "[/INST]"]},
    'model_name_or_path': "llm-stacking/StackLLM_410M_750BToken",
    'trust_remote_code': False,
    'dataset': "../data/preprocessed/wiki_auto_simp_new.json",
    'source_max_len': 512,
    'target_max_len': 512,
    'train_on_source': True,
    'predict_with_generate': False
    
}

# Convert dictionary to object
args = ArgsObject(**args_dict)

# Access values using dot notation
print(args.model_name_or_path)  # Output: llm-stacking/StackLLM_410M_750BToken
print(args.special_tokens_dict)  # Output: {'additional_special_tokens': ['<<SYS>>', '<</SYS>>', '[INST]', '[/INST]']}

llm-stacking/StackLLM_410M_750BToken
{'additional_special_tokens': ['<<SYS>>', '<</SYS>>', '[INST]', '[/INST]']}


In [76]:
def get_accelerate_model(args, checkpoint_dir):

    device_map = "auto"

    # if we are in a distributed setting, we need to set the device map and max memory per device
    if os.environ.get('LOCAL_RANK') is not None:
        local_rank = int(os.environ.get('LOCAL_RANK', '0'))
        device_map = {'': local_rank}


    print(f'loading base model {args.model_name_or_path}...')
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        device_map=device_map,
        trust_remote_code=args.trust_remote_code,
    )

    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        args.model_name_or_path,
        padding_side="right",
        use_fast=True, # Fast tokenizer giving issues.
        trust_remote_code=args.trust_remote_code,
    )
    if tokenizer._pad_token is None:
        special_tokens_dict = dict(pad_token=DEFAULT_PAD_TOKEN)
        if args.dataset == "OpenAssistant/oasst_top1_2023-08-25":
            chat_special_tokens = ["<|im_start|>", "<|im_end|>"]
            special_tokens_dict.update(additional_special_tokens=chat_special_tokens)
        special_tokens_dict.update(
            additional_special_tokens=['<<SYS>>', '<</SYS>>', '[INST]', '[/INST]']
        )        
        
        smart_tokenizer_and_embedding_resize(
            special_tokens_dict=special_tokens_dict,
            tokenizer=tokenizer,
            model=model
        )

    return model, tokenizer

In [77]:
model, tokenizer = get_accelerate_model(args, checkpoint_dir=None)

loading base model llm-stacking/StackLLM_410M_750BToken...


c:\Users\esual\miniconda3\envs\clearningNew\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at C:\Users\esual/.cache\huggingface\hub\models--llm-stacking--StackLLM_410M_750BToken\snapshots\78169017f443a06dd3b742bc501b6121c9ba1afa\config.json
Model config LlamaConfig {
  "_name_or_path": "llm-stacking/StackLLM_410M_750BToken",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 2816,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_key_value_heads": 16,
  "pad_token_id": 0,
  "pretraining_tp": 

Resized tokenizer and embedding to 32005 tokens.


In [13]:
tokenizer._pad_token

'[PAD]'

In [78]:
import torch
from transformers import AutoTokenizer

def preprocess_example(example, tokenizer, max_length=512):
    SYSTEM_PROMPT = (
        "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request."
    )
    normal_sentence = example['input']
    simple_sentence = example['label']

    # Create input text
    input_text = f"<<SYS>>\n{SYSTEM_PROMPT}\n<</SYS>>\n\n[INST]\n{normal_sentence}[/INST]\n"

    # Tokenize input and output separately to determine lengths
    input_tokens = tokenizer(input_text, truncation=True, max_length=max_length, return_tensors="pt")
    output_tokens = tokenizer(simple_sentence, truncation=True, max_length=max_length, return_tensors="pt")

    input_length = input_tokens['input_ids'].shape[1]
    output_length = output_tokens['input_ids'].shape[1]

    # Ensure the combined length does not exceed max_length
    if input_length + output_length > max_length:
        # Truncate the output tokens to fit within max_length
        output_length = max_length - input_length
        output_tokens = tokenizer(
            simple_sentence,
            truncation=True,
            max_length=output_length,
            return_tensors="pt"
        )

    # Create labels
    labels = torch.cat([
        torch.full((input_length,), -100),  # Ignore input tokens
        output_tokens['input_ids'].squeeze()  # Use output tokens as labels
    ])

    # Pad labels to max_length
    padding_length = max_length - len(labels)
    if padding_length > 0:
        labels = torch.cat([labels, torch.full((padding_length,), -100)])
    else:
        # If padding_length is negative, truncate labels to max_length
        labels = labels[:max_length]

    # Tokenize the combined sequence
    combined_text = input_text + simple_sentence
    tokenized_sequence = tokenizer(
        combined_text,
        truncation=True,
        max_length=max_length,
        padding="max_length",
        return_tensors="pt"
    )

    return {
        'input_ids': tokenized_sequence['input_ids'].squeeze(),
        'attention_mask': tokenized_sequence['attention_mask'].squeeze(),
        'labels': labels,
        'all_input': input_text
    }

In [79]:
train_dataset = full_dataset['train'].map(
    lambda x: preprocess_example(x, tokenizer),
    batched=False
)

eval_dataset = full_dataset['eval'].map(
    lambda x: preprocess_example(x, tokenizer),
    batched=False
)

test_dataset = full_dataset['test'].map(
    lambda x: preprocess_example(x, tokenizer),
    batched=False
)

Map: 100%|██████████| 8000/8000 [00:08<00:00, 960.26 examples/s] 


In [81]:
train_dataset['all_input']

['<<SYS>>\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n<</SYS>>\n\n[INST]\nMake this text simpler: "When World War One broke out , Ho ̈ ss served briefly in a military hospital and then , at age 14 , was admitted to his father \'s and grandfather \'s old regiment , the German Army \'s 21st Regiment of Dragoons .\n"[/INST]\n',
 '<<SYS>>\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n<</SYS>>\n\n[INST]\nMake this text simpler: "Roderick E. Liddle -LRB- born 1 April 1960 -RRB- is an English journalist , and an associate editor of " The Spectator " .\n"[/INST]\n',
 '<<SYS>>\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n<</SYS>>\n\n[INST]\nReformulate thi

In [25]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [41]:
from dataclasses import dataclass, field


@dataclass
class DataCollatorForCausalLM(object):
    tokenizer: transformers.PreTrainedTokenizer
    source_max_len: int
    target_max_len: int
    train_on_source: bool
    predict_with_generate: bool

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        # Extract elements
        sources = [f"{self.tokenizer.bos_token}{example['input']}" for example in instances]
        targets = [f"{example['labels']}{self.tokenizer.eos_token}" for example in instances]
        # Tokenize
        tokenized_sources_with_prompt = self.tokenizer(
            sources,
            max_length=self.source_max_len,
            truncation=True,
            add_special_tokens=False,
        )
        tokenized_targets = self.tokenizer(
            targets,
            max_length=self.target_max_len,
            truncation=True,
            add_special_tokens=False,
        )
        # Build the input and labels for causal LM
        input_ids = []
        labels = []
        for tokenized_source, tokenized_target in zip(
            tokenized_sources_with_prompt['input_ids'],
            tokenized_targets['input_ids']
        ):
            if not self.predict_with_generate:
                input_ids.append(torch.tensor(tokenized_source + tokenized_target))
                if not self.train_on_source:
                    labels.append(
                        torch.tensor([IGNORE_INDEX for _ in range(len(tokenized_source))] + copy.deepcopy(tokenized_target))
                    )
                else:
                    labels.append(torch.tensor(copy.deepcopy(tokenized_source + tokenized_target)))
            else:
                input_ids.append(torch.tensor(tokenized_source))
        # Apply padding
        input_ids = pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        labels = pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX) if not self.predict_with_generate else None
        data_dict = {
            'input_ids': input_ids,
            'attention_mask':input_ids.ne(self.tokenizer.pad_token_id),
        }
        if labels is not None:
            data_dict['labels'] = labels
        return data_dict

In [42]:
data_collator = DataCollatorForCausalLM(
        tokenizer=tokenizer,
        source_max_len=args.source_max_len,
        target_max_len=args.target_max_len,
        train_on_source=args.train_on_source,
        predict_with_generate=args.predict_with_generate,
    )

In [43]:
class CustomSeq2SeqTrainer(Seq2SeqTrainer):
    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        if self.args.predict_with_generate:
            # Perform generation with all relevant parameters
            generated_tokens = model.generate(
                inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=getattr(self.args, "generation_max_length", 256),
                num_beams=getattr(self.args, "num_beams", 1),
                temperature=getattr(self.args, "temperature", 1.0),
                top_k=getattr(self.args, "top_k", 50),
                top_p=getattr(self.args, "top_p", 0.9),
                length_penalty=getattr(self.args, "length_penalty", 1.0),  # Add length_penalty
                eos_token_id=self.tokenizer.eos_token_id,
            )
            labels = inputs.get("labels")
            return None, generated_tokens, labels
        else:
            # Default behavior for non-generation mode
            return super().prediction_step(model, inputs, prediction_loss_only, ignore_keys)

In [83]:
data_module = dict(
        train_dataset=train_dataset if True else None,
        eval_dataset=eval_dataset if True else None,
        predict_dataset=eval_dataset if False else None,
        data_collator=data_collator
    )

In [48]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
)

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [51]:
import numpy as np
import torch
import evaluate

# Load metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred, tokenizer, model):
    """
    Compute perplexity, BLEU, and ROUGE scores for evaluation.

    Args:
        eval_pred (tuple): A tuple containing `predictions` and `labels`.
        tokenizer (PreTrainedTokenizer): The tokenizer used for the model.
        model (PreTrainedModel): The model being evaluated.

    Returns:
        dict: A dictionary containing perplexity, BLEU, and ROUGE scores.
    """
    predictions, labels = eval_pred

    # Decode predictions and labels into text
    pred_texts = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute BLEU
    bleu_results = bleu.compute(predictions=pred_texts, references=[[ref] for ref in label_texts])

    # Compute ROUGE
    rouge_results = rouge.compute(predictions=pred_texts, references=label_texts)

    # Compute perplexity
    perplexities = []
    for pred in pred_texts:
        # Tokenize the prediction
        input_ids = tokenizer(pred, return_tensors="pt").input_ids
        with torch.no_grad():
            # Compute the loss for the prediction
            outputs = model(input_ids, labels=input_ids)
            loss = outputs.loss
            perplexity = torch.exp(loss).item()
        perplexities.append(perplexity)

    # Aggregate perplexity
    avg_perplexity = np.mean(perplexities)

    return {
        'perplexity': avg_perplexity,
        'bleu': bleu_results['bleu'],
        'rouge1': rouge_results['rouge1'],
        'rouge2': rouge_results['rouge2'],
        'rougeL': rouge_results['rougeL'],
    }

In [52]:
# Initialize the Trainer
trainer = CustomSeq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=lambda eval_pred: compute_metrics(eval_pred, tokenizer, model),
)

You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.


In [53]:
train_result = trainer.train()

The following columns in the training set don't have a corresponding argument in `LlamaForCausalLM.forward` and have been ignored: index, src_info, input. If index, src_info, input are not expected by `LlamaForCausalLM.forward`,  you can safely ignore this message.
c:\Users\esual\miniconda3\envs\clearningNew\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 95,001
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 71,253
  Number of trainable parameters = 373,869,568
  0%|          | 0/71253 [00:00<?, ?it/s]You're using a LlamaTokenizerFast tokenizer. Please note that 

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [54]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("llm-stacking/StackLLM_410M_750BToken")

# Check if "simplify" is in the tokenizer's vocabulary
word = "simplify"
if word in tokenizer.get_vocab():
    print(f"'{word}' is already in the tokenizer's vocabulary.")
else:
    print(f"'{word}' is NOT in the tokenizer's vocabulary.")

c:\Users\esual\miniconda3\envs\clearningNew\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading file tokenizer.model from cache at C:\Users\esual/.cache\huggingface\hub\models--llm-stacking--StackLLM_410M_750BToken\snapshots\78169017f443a06dd3b742bc501b6121c9ba1afa\tokenizer.model
loading file tokenizer.json from cache at C:\Users\esual/.cache\huggingface\hub\models--llm-stacking--StackLLM_410M_750BToken\snapshots\78169017f443a06dd3b742bc501b6121c9ba1afa\tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\esual/.cache\huggingface\hub\models--llm-stacking--StackLLM_410M_750BToken\snapshots\78169017f443a06dd3b742bc501b6121c9ba1afa\special_tokens_map.json
loading file tokenizer_config.json from cache at 

'simplify' is NOT in the tokenizer's vocabulary.


In [65]:
# Check if "simplify" is in the tokenizer's vocabulary
word = "reformulate"
if word in tokenizer.get_vocab():
    print(f"'{word}' is already in the tokenizer's vocabulary.")
else:
    print(f"'{word}' is NOT in the tokenizer's vocabulary.")

'bos_token' is NOT in the tokenizer's vocabulary.


In [66]:
tokenizer.special_tokens_map

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}

In [68]:
data_module['train_dataset']

Dataset({
    features: ['input', 'label', 'src_info', 'index', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 95001
})

In [70]:
smart_tokenizer_and_embedding_resize({'additional_special_tokens': ['<<SYS>>', '<</SYS>>', '[INST]', '[/INST]']}, tokenizer, model)

Assigning ['<<SYS>>', '<</SYS>>', '[INST]', '[/INST]'] to the additional_special_tokens key of the tokenizer


Resized tokenizer and embedding to 32004 tokens.


In [71]:
# Check if "simplify" is in the tokenizer's vocabulary
word = "<<SYS>>"
if word in tokenizer.get_vocab():
    print(f"'{word}' is already in the tokenizer's vocabulary.")
else:
    print(f"'{word}' is NOT in the tokenizer's vocabulary.")

'<<SYS>>' is already in the tokenizer's vocabulary.


In [73]:
data_module['train_dataset']['input']

['Make this text simpler: "When World War One broke out , Ho ̈ ss served briefly in a military hospital and then , at age 14 , was admitted to his father \'s and grandfather \'s old regiment , the German Army \'s 21st Regiment of Dragoons .\n"',
 'Make this text simpler: "Roderick E. Liddle -LRB- born 1 April 1960 -RRB- is an English journalist , and an associate editor of " The Spectator " .\n"',
 'Reformulate this text with simpler words: "Rachmaninoff wrote five works for piano and orchestra : four concertos — No. 1 in F-sharp minor , Op. 1 -LRB- 1891 , revised 1917 -RRB- , No. 2 in C minor , Op. 18 -LRB- 1900 – 01 -RRB- , No. 3 in D minor , Op. 30 -LRB- 1909 -RRB- , and No. 4 in G minor , Op. 40 -LRB- 1926 , revised 1928 and 1941 -RRB- — plus the " Rhapsody on a Theme of Paganini " .\n"',
 "While playing at the University of Minnesota - Duluth , Maciver earned a communications degree and signed as a free agent with the NHL 's New York Rangers upon graduation .\n\n\n===\n\nGiven the

In [87]:
tokenized_spec_false = tokenizer(
    data_module['train_dataset']['all_input'],
    truncation=True,
    max_length=512,
    add_special_tokens=False
)

In [89]:
len(tokenized_spec_false)

2

In [3]:
from evaluate import load
perplexity = load("perplexity", module_type="metric")
results = perplexity.compute(predictions=[], model_id='gpt2')

c:\Users\esual\miniconda3\envs\clearningNew\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\esual\miniconda3\envs\clearningNew\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\esual\miniconda3\envs\clearningNew\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\esual\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disable

IndexError: list index out of range

In [ ]:
"llm-stacking/StackLLM_410M_750BToken"

perplexity = load("perplexity", module_type="metric")
results = perplexity.compute(predictions=[], model_id="llm-stacking/StackLLM_410M_750BToken")